In [1]:
!nvidia-smi

In [2]:
#%load_ext cudf.pandas

# To desable GPU usage
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

#from cucim.skimage.exposure import rescale_intensity
import tensorflow as tf
#import cupy as cp
#import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
import gc

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # memory limit 16GB (16 * 1024 MB = 16384 MB) 
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=16384)] 
        )
        print("Set GPU memory limit to 16GB.")
    except RuntimeError as e:
        print("Error setting memory limit:", e)
else:
    print("No GPUs available.")

#print("Is torch using cuda? ",torch.cuda.is_available())
print("Is tensorflow using cuda? ",tf.test.is_built_with_cuda())
print("Is pandas using cuda? ",pd)


In [ ]:
name_mapping = [
    "box",
    "circularTorus",
    "cone",
    "coneOffset",
    "cylinder",
    "cylinderSlope",
    "dish",
    "mesh",
    "pyramid",
    "rectangularTorus",
    "sphere"
]

In [ ]:
num_classes = 11
def filter_invalid_labels(image, label):

    valid_labels = tf.reduce_all(label >= 0) & tf.reduce_all(label < num_classes)
    return valid_labels

In [5]:
!ls /home/workspace/geometry-classifier/data/

In [6]:
base_path = '/home/workspace/geometry-classifier/data/'

def parse_tfrecord(example_proto):

    feature_description = {
        'name': tf.io.FixedLenFeature([], tf.string),
        'image': tf.io.FixedLenFeature([], tf.string)
    }
    parsed_example = tf.io.parse_single_example(example_proto, feature_description)
    name = tf.io.decode_raw(parsed_example['name'], tf.float32)
    image = tf.io.decode_raw(parsed_example['image'], tf.float32)
    image = tf.reshape(image, (224, 224, 1))  # Reshape image data to 224x224x3
    class_label = tf.cast(name[0], tf.int32)  # Use the first byte as a class label
    return image, class_label

def load_tfrecord(tfrecord_files):

    raw_dataset = tf.data.TFRecordDataset(tfrecord_files, compression_type="GZIP")
    parsed_dataset = raw_dataset.map(parse_tfrecord)
    return parsed_dataset


files_for = lambda file_type: glob.glob(f"{base_path}/tfr_strat100-2min/{file_type}*.tfrecord.gz")

test_files = files_for("test")
validation_files = files_for("val")
train_files = files_for("train")


test = load_tfrecord(test_files).filter(filter_invalid_labels)
test = test.batch(32).prefetch(tf.data.AUTOTUNE)

validation = load_tfrecord(validation_files).filter(filter_invalid_labels)
validation = validation.batch(32).prefetch(tf.data.AUTOTUNE)

train = load_tfrecord(train_files).filter(filter_invalid_labels)
train = train.shuffle(23360).batch(32).prefetch(tf.data.AUTOTUNE)



In [7]:
# 224 x 224

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

model = models.Sequential([
    layers.Conv2D(8, (3, 3), activation='relu', input_shape=(224,224,1)),
    layers.MaxPooling2D((2, 2)),
    

    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    

    layers.Flatten(),
    
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(11, activation='softmax') 
])

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [8]:
checkpoint_callback = ModelCheckpoint(base_path+'model4.keras',save_best_only=True, save_weights_only=False, mode='min', verbose=1)
early_stopping = EarlyStopping(
    monitor="val_loss",   
    patience=10,
    restore_best_weights=True, 
    verbose=1
)

epochs = 100

In [ ]:

cnn = model.fit(train, epochs=epochs,callbacks=[
    early_stopping,
    checkpoint_callback
], validation_data=validation)

  14912/Unknown 2765s 184ms/step - accuracy: 0.4762 - loss: 1.5182

2024-11-27 15:56:09.420877: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 1: val_loss improved from inf to 1.16318, saving model to /home/workspace/geometry-classifier/data/model4.keras
14912/14912 ━━━━━━━━━━━━━━━━━━━━ 2986s 199ms/step - accuracy: 0.4763 - loss: 1.5182 - val_accuracy: 0.5934 - val_loss: 1.1632


2024-11-27 15:59:50.016695: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


Epoch 2/100


...

14912/14912 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.8178 - loss: 0.4944
Epoch 10: val_loss improved from 0.76208 to 0.75298, saving model to /home/workspace/geometry-classifier/data/model4.keras
14912/14912 ━━━━━━━━━━━━━━━━━━━━ 2940s 196ms/step - accuracy: 0.8178 - loss: 0.4944 - val_accuracy: 0.7588 - val_loss: 0.7530
Epoch 11/100


2024-11-27 23:26:35.426961: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:16: Filling up shuffle buffer (this may take a while): 12661 of 23360
2024-11-27 23:26:43.040428: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


14912/14912 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.8226 - loss: 0.4780
Epoch 11: val_loss improved from 0.75298 to 0.73113, saving model to /home/workspace/geometry-classifier/data/model4.keras
14912/14912 ━━━━━━━━━━━━━━━━━━━━ 2853s 190ms/step - accuracy: 0.8226 - loss: 0.4780 - val_accuracy: 0.7630 - val_loss: 0.7311
Epoch 12/100


2024-11-28 00:14:08.426592: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:16: Filling up shuffle buffer (this may take a while): 13456 of 23360
2024-11-28 00:14:15.236042: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


 8626/14912 ━━━━━━━━━━━━━━━━━━━━ 19:11 183ms/step - accuracy: 0.8278 - loss: 0.4577

In [ ]:
test_loss, test_accuracy = model.evaluate(test)
test_loss, test_accuracy

In [ ]:
history = cnn


plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix


y_true = []
y_pred = []

for images, labels in test:
    y_true.extend(labels.numpy())
    predictions = model.predict(images)
    y_pred.extend(np.argmax(predictions, axis=1))
    
y_true = np.array(y_true)
y_pred = np.array(y_pred)

In [ ]:


cm = confusion_matrix(y_true, y_pred)


plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=name_mapping, yticklabels=name_mapping)
plt.title('Confusion Matrix without normalization')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:

cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100


plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues', xticklabels=name_mapping, yticklabels=name_mapping)
plt.title('Confusion Matrix in porcentages')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()
